In [26]:
from scripts.convert_brat_annotations_to_json import load_jsonl, used_entities
def span_match(span_1, span_2) :
    sa, ea = span_1
    sb, eb = span_2
    iou = (min(ea, eb) - max(sa, sb)) / (max(eb, ea) - min(sa, sb))
    return iou

ents = used_entities

In [49]:
data = {}
data[0] = load_jsonl('../model_data/all_data_propagated.jsonl')[:440]
data[1] = load_jsonl('../model_data/all_data_ann_1_propagated.jsonl')[:440]
data[2] = load_jsonl('../model_data/all_data_ann_2_propagated.jsonl')[:440]
data[3] = load_jsonl('../model_data/all_data_ann_3_propagated.jsonl')[:440]
data[4] = load_jsonl('../model_data/all_data_ann_4_propagated.jsonl')[:440]

In [50]:
for d in data :
    data[d] = {x['doc_id']:x for x in data[d] if x['doc_id'] in 
               ['063ad0349f05c8aacbbb653ffcf01047a293fa30', '06b4d8409837dce9d6eb919efd1debdaecc40d01']}
    

In [51]:
token_level = {}
for d in data :
    token_level[d] = {x['doc_id']:(['None']*len(x['words'])) for x in data[d].values()}
    
    for x in data[d].values() :
        for e in x['ner'] :
            for i in range(e[0], e[1]) :
                token_level[d][x['doc_id']][i] = e[2]

In [52]:
from sklearn.metrics import cohen_kappa_score
from dygie.commands.generate_metrics import generate_metrics_for_ner

In [53]:
for doc in ['063ad0349f05c8aacbbb653ffcf01047a293fa30', '06b4d8409837dce9d6eb919efd1debdaecc40d01'] :
    print(doc)
    common = [0]*len(token_level[0][doc])
    val = token_level[0][doc]
    for i in range(4) :
        for j in range(4) :
            c = [1 if x != y else 0 for x, y in zip(token_level[i][doc], token_level[j][doc])]
            common = [max(x, y) for x, y in zip(c, common)]

    print(sum(common))

    for i in range(4) :
        for j in range(4) :
            print(cohen_kappa_score([y for x, y in zip(common, token_level[i][doc]) if x == 1], 
                                    [y for x, y in zip(common, token_level[j][doc]) if x == 1]))
        

063ad0349f05c8aacbbb653ffcf01047a293fa30
22
1.0
-0.1445086705202312
-0.26701570680628284
0.16551724137931034
-0.1445086705202312
1.0
0.674074074074074
0.4122137404580153
-0.26701570680628284
0.674074074074074
1.0
0.2576687116564418
0.16551724137931034
0.4122137404580153
0.2576687116564417
1.0
06b4d8409837dce9d6eb919efd1debdaecc40d01
70
1.0
-0.0758009898411045
-0.04644954618259489
-0.042183622828784184
-0.0758009898411045
1.0
0.7659732540861812
0.4534962661235574
-0.04644954618259489
0.7659732540861812
1.0
0.47725729803122885
-0.04218362282878396
0.4534962661235574
0.47725729803122885
1.0


In [55]:
for e in ents + ['None']:
    a = 0
    t = 0
    for doc in ['063ad0349f05c8aacbbb653ffcf01047a293fa30', '06b4d8409837dce9d6eb919efd1debdaecc40d01'] :
        for i in range(5) :
            for j in range(5) :
                if i < j: 
                    n = cohen_kappa_score([1 if x == e else 0 for x in token_level[i][doc]], 
                                          [1 if x == e else 0 for x in token_level[j][doc]])
                    a += n
                    t += 1
                    
    print(e, a/t)

Material 0.9570187258956369
Metric 0.9611375207707636
Task 0.9829060839191659
Method 0.9742606210776751
None 0.9824777372728333


In [56]:
for i in range(4) :
    for j in range(4) :
        if i < j :
            for d in data[i] :
                data[i][d]['gold_spans'] = {(x[0], x[1]):x[2] for x in data[i][d]['ner']}
                data[i][d]['predicted_spans'] = {(x[0], x[1]):x[2] for x in data[j][d]['ner']}

            metrics = generate_metrics_for_ner(data[i])
            print(metrics)
        

\begin{tabular}{lrrrrrr}
\toprule
{} &  Exact F1 &  Exact P &  Exact R &  Soft F1 &  Soft P &  Soft R \\
\midrule
Material &     0.974 &    1.000 &    0.950 &    0.974 &   1.000 &   0.950 \\
Task     &     0.937 &    0.901 &    0.977 &    0.945 &   0.908 &   0.984 \\
Method   &     0.913 &    0.916 &    0.910 &    0.923 &   0.927 &   0.920 \\
Metric   &     0.913 &    0.955 &    0.875 &    0.913 &   0.955 &   0.875 \\
\bottomrule
\end{tabular}

None
\begin{tabular}{lrrrrrr}
\toprule
{} &  Exact F1 &  Exact P &  Exact R &  Soft F1 &  Soft P &  Soft R \\
\midrule
Material &     0.974 &    1.000 &    0.950 &    0.974 &   1.000 &   0.950 \\
Task     &     0.908 &    0.872 &    0.948 &    0.945 &   0.908 &   0.984 \\
Method   &     0.913 &    0.916 &    0.910 &    0.923 &   0.927 &   0.920 \\
Metric   &     0.919 &    0.969 &    0.875 &    0.933 &   0.984 &   0.887 \\
\bottomrule
\end{tabular}

None
\begin{tabular}{lrrrrrr}
\toprule
{} &  Exact F1 &  Exact P &  Exact R &  Soft F1 &  Soft P 